In [1]:
import math
from sklearn import preprocessing

In [43]:

# we'll generate some test texts to experiment with
corpus = [
         "apple cannot patent the concept of a region Apple cannot patent"
         "the concept of a region Apple also has a patent on their ADB hardware"
         "apple cannot patent the concept of a region Apple cannot patent the concept" 
         "of a region Apple also has a patent on their hardware",
    
      "The BEST option is XFree86, which is an enhanced version of X386 1.2."
         "Any other version of X386 will have slower performance, and will"
          "be more difficult to compile.  Information on how to obtain XFree86"
          "is listed below.",
    
    
      "easily installed in existing pc clone devices, you mean"
         "and thats not even really true, cause neither the controller"
         "nor the computers bios will know anything about them"
        " and it probably wouldnt be in sony's (or whoever) best interest"
         "to restrict themselves to customers using pc clones and apple",
    
     " comp.sys.mac.hardware Appel, Mac,  computer, Macintosh ADB hardware "
    "is a proprietary bit-serial peripheral bus connecting low-speed devices to computers."
          " It was introduced on the Apple IIGS in 1986 as a way to support low-cost devices "
           "like keyboards and mice,"
          "allowing them to be connected together in a daisy chain without the need for hubs or other devices"
         
]

# remove stop words and tokenize them (we probably want to do some more
# preprocessing with our text in a real world setting, but we'll keep
# it simple here)
stopwords = set(['for', 'a', 'of', 'the', 'and', 'to', 'in'])
texts = [
    [word for word in document.lower().split() if word not in stopwords]
    for document in corpus
]

# build a word count dictionary so we can remove words that appear only once
word_count_dict = {}
for text in texts:
    for token in text:
        word_count = word_count_dict.get(token, 0) + 1
        word_count_dict[token] = word_count

texts = [[token for token in text if word_count_dict[token] > 0] for text in texts]
texts

[['apple',
  'cannot',
  'patent',
  'concept',
  'region',
  'apple',
  'cannot',
  'patentthe',
  'concept',
  'region',
  'apple',
  'also',
  'has',
  'patent',
  'on',
  'their',
  'adb',
  'hardwareapple',
  'cannot',
  'patent',
  'concept',
  'region',
  'apple',
  'cannot',
  'patent',
  'conceptof',
  'region',
  'apple',
  'also',
  'has',
  'patent',
  'on',
  'their',
  'hardware'],
 ['best',
  'option',
  'is',
  'xfree86,',
  'which',
  'is',
  'an',
  'enhanced',
  'version',
  'x386',
  '1.2.any',
  'other',
  'version',
  'x386',
  'will',
  'have',
  'slower',
  'performance,',
  'willbe',
  'more',
  'difficult',
  'compile.',
  'information',
  'on',
  'how',
  'obtain',
  'xfree86is',
  'listed',
  'below.'],
 ['easily',
  'installed',
  'existing',
  'pc',
  'clone',
  'devices,',
  'you',
  'meanand',
  'thats',
  'not',
  'even',
  'really',
  'true,',
  'cause',
  'neither',
  'controllernor',
  'computers',
  'bios',
  'will',
  'know',
  'anything',
  'about

In [44]:
class BM25:
    """
    Best Match 25.

    Parameters
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    Attributes
    ----------
    tf_ : list[dict[str, int]]
        Term Frequency per document. So [{'hi': 1}] means
        the first document contains the term 'hi' 1 time.

    df_ : dict[str, int]
        Document Frequency per term. i.e. Number of documents in the
        corpus that contains the term.

    idf_ : dict[str, float]
        Inverse Document Frequency per term.

    doc_len_ : list[int]
        Number of terms per document. So [3] means the first
        document contains 3 terms.

    corpus_ : list[list[str]]
        The input corpus.

    corpus_size_ : int
        Number of documents in the corpus.

    avg_doc_len_ : float
        Average number of terms for documents in the corpus.
    """

    def __init__(self, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1

    def fit(self, corpus):
        """
        Fit the various statistics that are required to calculate BM25 ranking
        score using the corpus given.

        Parameters
        ----------
        corpus : list[list[str]]
            Each element in the list represents a document, and each document
            is a list of the terms.

        Returns
        -------
        self
        """
        tf = []
        df = {}
        idf = {}
        doc_len = []
        corpus_size = 0
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))

            # compute tf (term frequency) per document
            frequencies = {}
            for term in document:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count

            tf.append(frequencies)

            # compute df (document frequency) per term
            for term, _ in frequencies.items():
                df_count = df.get(term, 0) + 1
                df[term] = df_count

        for term, freq in df.items():
            idf[term] = math.log(1 + (corpus_size - freq + 0.5) / (freq + 0.5))

        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size
        return self

    def search(self, query):
        scores = [self._score(query, index) for index in range(self.corpus_size_)]
        return scores

    def _score(self, query, index):
        score = 0.0

        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term not in frequencies:
                continue

            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len_)
            score += (numerator / denominator)

        return score

In [50]:
# query our corpus to see which document is more relevant
query = 'Computer bus'
query = [word for word in query.lower().split() if word not in stopwords]

bm25 = BM25()
bm25.fit(texts)
scores = bm25.search(query)

for score, doc in zip(scores, corpus):
    score = round(score, 3)
        

    print(str(score) + '\t' + doc)
       


0.0	apple cannot patent the concept of a region Apple cannot patentthe concept of a region Apple also has a patent on their ADB hardwareapple cannot patent the concept of a region Apple cannot patent the conceptof a region Apple also has a patent on their hardware
0.0	The BEST option is XFree86, which is an enhanced version of X386 1.2.Any other version of X386 will have slower performance, and willbe more difficult to compile.  Information on how to obtain XFree86is listed below.
0.0	easily installed in existing pc clone devices, you meanand thats not even really true, cause neither the controllernor the computers bios will know anything about them and it probably wouldnt be in sony's (or whoever) best interestto restrict themselves to customers using pc clones and apple
1.107	 comp.sys.mac.hardware Appel, Mac,  computer, Macintosh ADB hardware is a proprietary bit-serial peripheral bus connecting low-speed devices to computers. It was introduced on the Apple IIGS in 1986 as a way to 

In [20]:
query2 = 'macintosh hardware'
query2= [word for word in query2.lower().split() if word not in stopwords]
scores2 = bm25.search(query2)

In [21]:
for score2, doc in zip(scores2, corpus):
    score2 = round(score2, 3)
        

    print(str(score2) + '\t' + doc)

0.662	macintosh intel mac apple cannot patent the concept of a region Apple cannot patent the concept of a region Apple also has a patent on their ADB hardware
0.0	apple cannot patent the concept of a region Apple cannot patent the concept of a region Apple also has a patent on their ADB hardware


In [22]:
query3 = 'Android hardware'
query3= [word for word in query3.lower().split() if word not in stopwords]
scores3 = bm25.search(query2)

In [23]:
for score3, doc in zip(scores3, corpus):
    score3 = round(score3, 3)
        

    print(str(score2) + '\t' + doc)

0.0	macintosh intel mac apple cannot patent the concept of a region Apple cannot patent the concept of a region Apple also has a patent on their ADB hardware
0.0	apple cannot patent the concept of a region Apple cannot patent the concept of a region Apple also has a patent on their ADB hardware
